![](introduccion.PNG)

![](Pregunta1.PNG)

In [43]:
def caculaTotales(categoria): # 'Flores'
    totales = {}
    f = open(categoria+'.txt', 'r')
    f.readline()
    for linea in f:
        comprador,vendedor,producto,Unidadesvendidas,ventas,Fecha = linea.strip().split(',')
        if (comprador,vendedor,producto) not in totales:
            totales[(comprador,vendedor,producto)] = float(ventas)
        else:
            totales[(comprador,vendedor,producto)] += float(ventas)
    return totales


![](Pregunta2.PNG)

In [51]:
def consolidado(nomArchivo, categorias): # 'Datos.csv' ['Maderas','Flores','Frutas']
    dicCategorias = {}
    f = open(nomArchivo,'w')
    for categoria in categorias: # MAderas
        totales = caculaTotales(categoria)
        productos = []
        for (comprador,vendedor,producto),ventas in totales.items():
            if producto not in productos:
                productos.append(producto)
            linea = comprador+','+vendedor+','+categoria+','+producto+','+str(ventas)+'\n'
            f.write(linea)
        dicCategorias[categoria] = productos
    f.close()
    for categoria in dicCategorias: # Flores
        dicCategorias[categoria].sort()
    return dicCategorias

In [61]:
def l_productos(dicCategorias, categorias):
    productos = []
    for categoria in categorias:
        productos_cate = dicCategorias[categoria]
        productos.extend(productos_cate)
    return productos  

In [62]:
def crearDicc(nomArchivo): # 'Datos.csv'
    dicc = {}
    f = open(nomArchivo, 'r')
    for linea in f: # Ecuador,Alemania,Flores,Narcisos,4523.18
        comprador,vendedor,categoria,producto,ventas = linea.strip().split(',')
        if categoria not in dicc:
            dicc[categoria]  = {(comprador, producto) : float(ventas)}
        else:
            if (comprador, producto) not in dicc[categoria]:
                dicc[categoria][(comprador, producto)] = float(ventas)
            else:
                dicc[categoria][(comprador, producto)] += float(ventas)
    f.close()
    return dicc     

In [63]:
def crearMatriz(nomArchivo, dicCategorias, compradores, categorias):
    productos = l_productos(dicCategorias, categorias)
    fils = len(compradores)
    cols = len(productos)
    M = np.zeros((fils,cols))
    dicc = crearDicc(nomArchivo)
    for categoria in categorias:
        for (comprador, producto), ventas in dicc[categoria].items():
            ind_fila = compradores.index(comprador)
            ind_colu = productos.index(producto)
            M[ind_fila, ind_colu] = ventas
    return M, productos

![](Pregunta3.PNG)

![](ProgramaPrincipal.PNG)

Finalmente, cree un programa principal para responder a las siguientes preguntas:

In [64]:
import numpy as np

In [65]:
def l_compradores(categorias):
    compradores = []
    for categoria in categorias:
        f = open(categoria + '.txt', 'r')
        f.readline()
        for linea in f:
            datos = linea.strip().split(',') 
            comprador = datos[0]
            if comprador not in compradores:
                compradores.append(comprador)
    return compradores

In [66]:
categorias = ['Maderas', 'Frutas', 'Flores']
nomArchivo = 'Datos.csv'
dicCategorias= consolidado(nomArchivo, categorias)
compradores = l_compradores(categorias)
M, productos = crearMatriz(nomArchivo, dicCategorias, compradores, categorias)

1. Cuál es el pais con mas ventas en total?

In [94]:
# iterar la matriz por fila
filas = M.shape[0] # 10
total_pais = np.zeros(filas) # [0 0 0 0 0 0 0 0 ... ]
for i in range(filas): # [0,1 ,2...9]
    fila  = M[i, :]
    suma = fila.sum()
    total_pais[i] = suma    
# cual es el mayor 
ind = np.argmax(total_pais)
mayor_pais = compradores[ind]
print('el pais con mas ventas en total es {}'.format(mayor_pais))

el pais con mas ventas en total es Rusia


2. Cual es el producto con menos ventas en total y a que categoria pertence?

In [95]:
def categoria_producto(dicCategorias, producto):
    for categoria,l_productos in dicCategorias.items():
        if producto in l_productos:
            return categoria

In [96]:
# iterar la matriz por columna
colus = M.shape[1]
v_prds_total = np.zeros(colus)
for j in range(colus):
    colu = M[:,j]
    total = colu.sum()
    v_prds_total[j] = total
ind_min = np.argmin(v_prds_total)
producto_menor = productos[ind_min]
cate_prod = categoria_producto(dicCategorias, producto_menor)
print(producto_menor)
print(cate_prod)

Abeto
Maderas


3. Top 5 de los países con más ventas considerando todos los productos.

In [97]:
total_pais

array([2969415.37, 3135062.07, 3378674.53, 2665481.13, 3737620.97,
       4024418.21, 2402009.56, 2071550.8 , 3672705.14, 2891092.69])

In [99]:
# top en listas
l_tuplas = []
for i in range(len(total_pais)):
    pais = compradores[i]
    ventas_pais = total_pais[i]
    tupla = (ventas_pais, pais)
    l_tuplas.append(tupla)
l_tuplas.sort(reverse = True)
top5 = l_tuplas[:5]
top5

[(4024418.2100000004, 'Alemania'),
 (3737620.9699999997, 'Italia'),
 (3672705.1400000006, 'China'),
 (3378674.53, 'Colombia'),
 (3135062.0700000003, 'Peru')]

In [106]:
# top con numpy
top5_pais = total_pais.argsort()[::-1][:5]
top5_pais

array([5, 4, 8, 2, 1], dtype=int64)

In [107]:
v_compradores = np.array(compradores)
v_compradores[top5_pais]

array(['Alemania', 'Italia', 'China', 'Colombia', 'Peru'], dtype='<U14')

In [81]:
l_tuplas

[(1736308.0299999998, 'Corazon Sangrante'),
 (1733278.35, 'Cedro'),
 (1680456.36, 'Margaritas'),
 (1496277.33, 'Papaya'),
 (1451324.06, 'Roble'),
 (1390944.69, 'Manzana'),
 (1282728.0, 'Naranja'),
 (1215744.9400000002, 'Arce'),
 (1156863.54, 'Dalias'),
 (1119902.6300000001, 'Narcisos'),
 (1118244.42, 'Kaki'),
 (1098933.8099999998, 'Ciclamen'),
 (1050320.35, 'Frambuesa'),
 (1045616.75, 'Rosas'),
 (1001147.6499999999, 'Fresno'),
 (961579.62, 'Mandarina'),
 (915047.62, 'Grosella'),
 (913676.4299999999, 'Cacao'),
 (891065.75, 'Peonias'),
 (861529.87, 'Haya'),
 (830185.8799999999, 'Cereza'),
 (794714.58, 'Nogal'),
 (794697.21, 'Caoba'),
 (784084.6700000002, 'Cerezo'),
 (731311.39, 'Azaleas'),
 (722247.87, 'Tulipanes'),
 (704273.83, 'Pino'),
 (650857.37, 'Limon'),
 (511006.91000000003, 'Lirios'),
 (303660.56000000006, 'Abeto')]

4. Solicite al usuario el nombre de un producto y muestre por pantalla los países ordenados acuerdo a las ventas para ese producto.

In [109]:
nombre_producto = input('Ingrese el nombre un producto ')

Ingrese el nombre un productoAbeto


In [110]:
ind_col = productos.index(nombre_producto)
v_producto = M[:,ind_col]
v_producto
v_inds = v_producto.argsort()[::-1]

In [113]:
v_compradores[v_inds]

array(['Rusia', 'Italia', 'China', 'Colombia', 'Alemania',
       'Estados Unidos', 'Peru', 'Ecuador', 'Francia', 'Mexico'],
      dtype='<U14')

5. Solicite al usuario un pais y muestre por pantalla que productos *no a comprado* en cada categoria-

In [114]:
pais = input('Ingrese un pais ')

Ingrese un pais Peru


In [117]:
# iterar una matriz por bloque
ini_col = 0
for categoria in categorias:
    fin_col = ini_col + len(dicCategorias[categoria])
    m_categoria = M[:,ini_col: fin_col]
    print(categoria)
    print(m_categoria)
    ini_col = fin_col

In [ ]:
ind_pais = compradores.index(pais)
ini_col = 0
for categoria in categorias:
    fin_col = ini_col + len(dicCategorias[categoria])
    prods_cate = np.array(productos[ini_col: fin_col])
    m_categoria = M[ind_pais,ini_col: fin_col]
    condicion = m_categoria == 0
    print(categoria)
    print(prods_cate[condicion])
    ini_col = fin_col

6. Cuál es la categoria con más ventas.

In [120]:
# iterar una matriz por bloque
ini_col = 0
cate_T = []
for categoria in categorias:
    fin_col = ini_col + len(dicCategorias[categoria])
    m_categoria = M[:,ini_col: fin_col]
    suma = m_categoria.sum()
    cate_T.append(suma)
    ini_col = fin_col
print(cate_T)
print(categorias)
v_cate_T = np.array(cate_T)
v_categorias = np.array(categorias)
cate_mayor = v_categorias[np.argmin(cate_T)]
cate_mayor

[9644455.72, 10609861.71, 10693713.04]
['Maderas', 'Frutas', 'Flores']


'Maderas'


7. Para cada pais muestre por pantalla con que productos de categoria Frutas superan a su producto más vendido en la categoria Flores. Si un pais no tiene ningun producto de Frutas que cumpla la condicion debe mostrar por pantalla un mensaje adecuado.


In [121]:
def matriz_Categoria(M,dicCategorias,categorias, nomcategoria):
    # iterar una matriz por bloque
    ini_col = 0
    for categoria in categorias:
        fin_col = ini_col + len(dicCategorias[categoria])
        if categoria == nomcategoria:
            m_categoria = M[:,ini_col: fin_col]
            return m_categoria
        ini_col = fin_col

In [126]:
m_Flores = matriz_Categoria(M,dicCategorias,categorias, 'Flores')
m_Flores

array([[     0.  , 100541.68, 473287.04, 166199.86,      0.  ,      0.  ,
         38707.69,      0.  , 283804.62,      0.  ],
       [259464.47,  28525.65,  85025.77,  92134.21, 143323.39,  64546.41,
         70636.36,      0.  ,  76769.99,      0.  ],
       [113019.48, 261862.88, 198866.07,  14606.85, 138183.95, 166164.87,
          1671.13, 126325.59,  76166.63, 203473.4 ],
       [     0.  ,  69799.35,      0.  ,  23942.69, 126927.19,      0.  ,
        264691.47,      0.  ,      0.  ,      0.  ],
       [ 41667.34, 108641.83,      0.  ,  33088.67,  13630.6 , 416831.13,
        189078.16, 264523.19,      0.  , 172353.71],
       [164167.44,  79202.5 , 495600.93, 194193.56,  18186.4 , 328188.62,
        213086.14, 122287.46,  61174.45, 162760.35],
       [ 51377.33,  71766.98, 265443.61,  35007.93,      0.  , 240162.86,
        109402.51,  57904.68,  29293.57,  42677.94],
       [  9053.84,      0.  ,      0.  ,  73381.88,      0.  ,  92768.84,
             0.  ,      0.  , 157285.

In [132]:
max_Flores = m_Flores.max(axis = 1)

In [131]:
m_Frutas = matriz_Categoria(M,dicCategorias,categorias, 'Frutas')
m_Frutas

array([[150398.65,      0.  , 107595.04, 173163.25,      0.  ,      0.  ,
        215631.53,      0.  , 329251.02,  45391.11],
       [204520.35,  86941.53,  84749.42, 269919.62, 147610.63,      0.  ,
          5427.37, 196812.34,  84815.93, 258909.92],
       [ 52753.65,  95323.67, 214239.43,      0.  , 228597.4 ,      0.  ,
         56856.75, 131060.77,  93743.34, 117118.6 ],
       [136497.85,  13681.29, 211485.56, 219972.24, 174445.35, 152721.77,
        203544.86,      0.  ,   5607.74, 257595.56],
       [164235.84, 141349.9 ,  16356.79,  33785.8 ,  16348.8 ,      0.  ,
        146182.53, 130169.83, 193870.98, 132210.71],
       [ 28296.4 ,  39340.56, 217142.92, 183427.91,  67773.35, 291424.92,
        137680.31,  89707.81, 164059.12, 135945.23],
       [100409.76, 130123.41,      0.  ,  34778.8 ,   9312.98,  30856.96,
          8288.56, 267408.27,  40494.81,  78940.11],
       [     0.  ,      0.  , 172699.4 ,      0.  , 148841.72,  72617.63,
         25176.32, 401200.83, 113073.

In [137]:
m_condicion = m_Frutas > max_Flores

In [140]:
v_prods_frutas = np.array(dicCategorias['Frutas'])
for i in range(len(compradores)):
    fila = m_condicion[i,:]#[False, False, False, False, False, False, False, False, False, False]
    v_prods_frutas[fila]
    print(compradores[i])
    if len(v_prods_frutas[fila])==0:
        print('no hay')
    else:
        print(v_prods_frutas[fila])
    

Rusia
no hay
Peru
['Grosella' 'Manzana']
Colombia
no hay
Mexico
no hay
Italia
no hay
Alemania
no hay
Ecuador
['Manzana']
Estados Unidos
['Manzana']
China
['Cereza' 'Papaya']
Francia
no hay
